In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

<font size="5" color="red">ch11. 데이터프레임과 시리즈(Pandas)_9절~ </font>
# 9절. 데이터 그룹화 및 집계

In [2]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Groupby
cf.pivot_table

In [3]:
# 1) 그룹화 방법1
#     df.groupby(by=[그룹핑할 열])[함수 적용해서 출력할 열들].기초 통계 함수()
#             [함수적용해서 출력할 열들] 생략시 그룹핑할 열을 제외한 모든 열 적용
iris_grouped = iris.groupby(iris.Species)
iris_grouped = iris.groupby(['Species'])
iris_grouped = iris.groupby('Species')
for group, data in iris_grouped:
    print(group)
    display(data.sample(2).sort_index()) # reset_index()

setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
18,5.7,3.8,1.7,0.3,setosa
36,5.5,3.5,1.3,0.2,setosa


versicolor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
82,5.8,2.7,3.9,1.2,versicolor
86,6.7,3.1,4.7,1.5,versicolor


virginica


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
101,5.8,2.7,5.1,1.9,virginica
139,6.9,3.1,5.4,2.1,virginica


In [4]:
iris.groupby(by='Species').mean()[['Petal.Length', 'Petal.Width']]
iris.groupby(by='Species')[['Petal.Length', 'Petal.Width']].mean()

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [5]:
iris_grouped_sum = iris.groupby(by='Species').median()
# iris_grouped_sum.reset_index()
iris_grouped_sum

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


In [6]:
iris_grouped_sum.loc[['setosa','versicolor']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [7]:
iris_grouped_sum2 = iris.groupby('Species', as_index=False).sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [8]:
iris_grouped_sum2[(iris_grouped_sum2.Species=='setosa') 
                  | (iris_grouped_sum2.Species=='versicolor')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


In [9]:
#     df.groupby(by=[그룹핑할 열])[함수 적용해서 출력할 열들].기초 통계 함수()
# 1) 그룹화 방법2
#     df.pivot_table(index=[그룹핑 할 열],
#                   values=[함수적용해서 출력할 열들],
#                     aggfunc=함수(기본값:mean))

In [10]:
# iris.groupby(by='Species')[['Petal.Length','Petal.Width']].mean()
iris.pivot_table(index='Species',
                values=['Petal.Length','Petal.Width'],
                aggfunc='mean')

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [92]:
iris.groupby('Species').count() # 결측치를 제외한 값의 개수
iris.pivot_table(index='Species',
                aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width,target
Species,,,,,
setosa,50,48,50,49,50
versicolor,49,49,49,49,50
virginica,48,49,50,50,50


In [12]:
iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


### 다중열로 그룹화

In [13]:
# iris 다중열로 그룹화 하기 위해 데이터 컬럼 추가(num 필드)
iris.shape

(150, 5)

In [14]:
import numpy as np
np.ravel([[i]*25 for i in range(6)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [15]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.iloc[::25]
# iris.groupby("num").count()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


In [16]:
# 각 열의 임의의 데이터에 결측치를 할당
# iris.iloc[0, 0] = np.nan

In [17]:
import random
print(random.sample(range(150), 3))
for col in range(iris.shape[1]-2) : # iris.shape[1]-2 : 4
    iris.iloc[random.sample(range(150), col+1), col] = np.nan

[95, 107, 1]


In [18]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  149 non-null    float64
 1   Sepal.Width   148 non-null    float64
 2   Petal.Length  147 non-null    float64
 3   Petal.Width   146 non-null    float64
 4   Species       150 non-null    object 
 5   num           150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [19]:
iris.groupby(by=['Species', 'num']).count()

Sepal.Length  Sepal.Width  Petal.Length  Petal.Width
Species    num                                                      
setosa     0              25           25            25           25
           1              25           24            25           23
versicolor 2              25           25            24           25
           3              24           24            25           24
virginica  4              25           25            24           24
           5              25           25            24           25

In [20]:
iris.groupby(by=['Species', 'num'],
            # as_index=False
            )[['Petal.Length', 'Petal.Width']].count()

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           25
           1              25           23
versicolor 2              24           25
           3              25           24
virginica  4              24           24
           5              24           25

In [168]:
iris.pivot_table(index=['Species', 'num'],
                values = ['Petal.Length', 'Petal.Width'],
                aggfunc='count')

KeyError: 'num'

In [24]:
iris.groupby('Species').describe().T

Species                setosa  versicolor  virginica
Sepal.Length count  50.000000   49.000000  50.000000
             mean    5.006000    5.934694   6.588000
             std     0.352490    0.521437   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
Sepal.Width  count  49.000000   49.000000  50.000000
             mean    3.426531    2.773469   2.974000
             std     0.382849    0.316080   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.500000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.700000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
Petal.Length count  50.000000   49.000000  48.000000
             mean    1.462000    4.261224   5.533333
             std     0.173664    0.474700   0.555182
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.400000   5.500000
             75%     1.575000    4.600000   5.800000
             max     1.900000    5.100000   6.900000
Petal.Width  count  48.000000   49.000000  49.000000
             mean    0.247917    1.324490   2.030612
             std     0.107168    0.199510   0.275533
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000
num          count  50.000000   50.000000  50.000000
             mean    0.500000    2.500000   4.500000
             std     0.505076    0.505076   0.505076
             min     0.000000    2.000000   4.000000
             25%     0.000000    2.000000   4.000000
             50%     0.500000    2.500000   4.500000
             75%     1.000000    3.000000   5.000000
             max     1.000000    3.000000   5.000000

## 9.2 레이블 인코딩

In [32]:
# iris.drop('num', axis=1, inplace=True)
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,NaN,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


In [38]:
# 레이블 인코딩을 지원하는 클래스 import
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # 객체
iris['target'] = le.fit_transform(iris.Species)
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,target
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,NaN,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절. 데이터 구조 변경
- 와이드 포맷 <-> 롱포맷
- 판다스 공식 문서의 melt :
https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
- Tidy Data(7Page) : https://vita.had.co.nz/papers/tidy-data.pdf

In [43]:
from statsmodels.api import datasets
airquality = datasets.get_rdataset('airquality',
                                  package='datasets').data
airquality.head(2)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2


## 10.1 melt를 이용한 언피벗팅(와이드포맷 -> 롱포맷)

In [89]:
air_melted = airquality.melt(id_vars=['Month', 'Day'],
               var_name='변수명', # 기본값:variable
               value_name='값') # 기본값 : value
air_melted.iloc[::20]
air_melted.sort_values(by=['Month', 'Day'], inplace=True)
air_melted

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0
...,...,...,...,...
610,9,29,Temp,76.0
152,9,30,Ozone,20.0
305,9,30,Solar.R,223.0
458,9,30,Wind,11.5


In [55]:
# melt 연습하기 (부동산 와이드포맷 데이터를 롱포맷) 
%ls C:\ai_x\download\shareData\부동산_250213\

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3664-591E

 C:\ai_x\download\shareData\부동산_250213 디렉터리

2025-04-30  오후 02:52    <DIR>          .
2025-04-30  오후 02:52    <DIR>          ..
2021-01-02  오후 03:44             2,163 전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv
2025-04-30  오후 02:04           419,340 주택도시보증공사_전국 신규 민간아파트 분양가격 동향_20241231.csv
               2개 파일             421,503 바이트
               2개 디렉터리  162,379,968,512 바이트 남음


In [86]:
import pandas as pd
df = pd.read_csv(r'C:\ai_x\download\shareData\부동산_250213\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv'
                ,encoding='cp949')
new_df = df.melt(id_vars="지역",
       var_name="연도월",
       value_name="평당분양가"
       )
new_df.sample()

지역          17
2013년12월    17
2014년1월     17
2014년2월     17
2014년3월     17
2014년4월     17
2014년5월     17
2014년6월     17
2014년7월     17
2014년8월     17
2014년9월     17
2014년10월    17
2014년11월    17
2014년12월    17
2015년1월     17
2015년2월     17
2015년3월     17
2015년4월     17
2015년5월     17
2015년6월     17
2015년7월     17
2015년8월     17
dtype: int64

## 10.2 pivot_table을 이용한 피벗팅(롱포맷 -> 와이드포맷)

In [2]:
# air_melted를 와이드 포맷으로
air_melted.head()

NameError: name 'air_melted' is not defined

In [1]:
air_quality2 = air_melted.pivot_table(index=['Month', 'Day'],
                      columns='변수명',
                      values='값')
air_quality2.reset_index(inplace=True)
air_quality2.columns.name = None

NameError: name 'air_melted' is not defined

In [90]:
air_quality2

,Month,Day,Ozone,Solar.R,Temp,Wind
0,5,1,41.0,190.0,67.0,7.4
1,5,2,36.0,118.0,72.0,8.0
2,5,3,12.0,149.0,74.0,12.6
3,5,4,18.0,313.0,62.0,11.5
4,5,5,NaN,NaN,56.0,14.3
...,...,...,...,...,...,...
148,9,26,30.0,193.0,70.0,6.9
149,9,27,NaN,145.0,77.0,13.2
150,9,28,14.0,191.0,75.0,14.3
151,9,29,18.0,131.0,76.0,8.0


# 11절. 데이터프레임(시리즈)에 함수 적용 시키기
- apply(func, axis) : 축 요소별(요소별) 함수 적용(데이터프레임이나 시리즈에서 가능)
- applymap(func) : 각 요소별 함수 적용(데이터 프레임에서만 가능)
- map(func) : 각 요소별 함수 적용(1차원 리스트, 시리즈에서만 가능)
## 11.1 apply

In [95]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [99]:
def get_year(datestr) : 
    return int(datestr.split('-')[0])

In [101]:
get_year = lambda datestr : int(datestr.split('-')[0])

In [102]:
get_year('1999-05-01')

1999

In [103]:
get_month = lambda datestr : int(datestr.split('-')[1])

In [104]:
get_date = lambda datestr : int(datestr.split('-')[2])

In [105]:
member['year'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[0]))
member.head()

,Name,Age,Email,Address,Birth,year
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999


In [108]:
X = iris.iloc[:, :-2]
# X의 열별 평균
avg = X.mean(axis=0)
avg

Sepal.Length    5.842282
Sepal.Width     3.057432
Petal.Length    3.724490
Petal.Width     1.207534
dtype: float64

In [111]:
X.iloc[0] - avg

Sepal.Length   -0.742282
Sepal.Width     0.442568
Petal.Length   -2.324490
Petal.Width    -1.007534
dtype: float64

In [115]:
# X의 모든 행에 대하여 열별 평균과의 거리를 소수점 2자리까지 출력
X.apply(lambda x: round(x-avg, 2), axis=1) # 람다 함수를 행에 적용시킨 것을 데이터프레임의 index만큼 반복함

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,-0.74,0.44,-2.32,-1.01
1,-0.94,-0.06,-2.32,-1.01
2,-1.14,0.14,-2.42,-1.01
3,-1.24,0.04,-2.22,-1.01
4,-0.84,0.54,-2.32,-1.01
...,...,...,...,...
145,0.86,-0.06,1.48,1.09
146,0.46,-0.56,1.28,0.69
147,0.66,-0.06,1.48,0.79
148,0.36,0.34,1.68,1.09


## 11.2 applymap

In [116]:
X.applymap(np.round)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,5.0,4.0,1.0,0.0
1,5.0,3.0,1.0,0.0
2,5.0,3.0,1.0,0.0
3,5.0,3.0,2.0,0.0
4,5.0,4.0,1.0,0.0
...,...,...,...,...
145,7.0,3.0,5.0,2.0
146,6.0,2.0,5.0,2.0
147,6.0,3.0,5.0,2.0
148,6.0,3.0,5.0,2.0


## 11.3 map
- 1차원

In [123]:
member['Month'] = member['Birth'].map(lambda datestr : int(datestr.split('-')[1]))
member

,Name,Age,Email,Address,Birth,year,Month
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1


In [136]:
# 연습하기(apply, map)
# new_df의 연도월 데이터를 연도와 월을 int로 분리하여 컬럼을 추가하세요
new_df["연도"] = new_df["연도월"].apply(lambda datestr : int(datestr.split('년')[0]))
new_df["월"] = new_df["연도월"].apply(lambda datestr : int(datestr.split("년")[1][:-1]))
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      357 non-null    object
 1   연도월     357 non-null    object
 2   평당분양가   357 non-null    int64 
 3   연도      357 non-null    int64 
 4   월       357 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 14.1+ KB


In [137]:
new_df.sample()

,지역,연도월,평당분양가,연도,월
236,경남,2015년1월,7592,2015,1


In [149]:
# new_df.drop(['연도','월'], axis=1, inplace=True)
new_df.head()

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098


In [ ]:
# ★ 2차원 데이터 프레임의 apply 이용하는 방법

In [163]:
def year_month_append(df) :
    'df를 받아 연도와 월 추가 후 return'
    df = df.copy()
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1][:-1])
    return df

In [164]:
year_month_append(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
연도           2013
월              12
Name: 0, dtype: object

In [167]:
new_df = new_df.apply(year_month_append, axis=1)
new_df

,지역,연도월,평당분양가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12
...,...,...,...,...,...
352,전북,2015년8월,6580,2015,8
353,전남,2015년8월,6289,2015,8
354,경북,2015년8월,7037,2015,8
355,경남,2015년8월,7665,2015,8


# 12절. 일괄 변경하기(결측치, 특정값)
## 12.1 fillna
- 대부분 결측치는 drop하거나, 평균값 대체, 중위수 대체, 그룹별 평균값, 그룹별 중위수, 이전값, 이후값 등으로 대체

In [63]:
import pandas as pd
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                  [3,       4, np.nan, 1],
                  [np.nan,  3, np.nan, 5],
                  [np.nan,  3, np.nan, 4]],
                 columns=[list('ABCD')])
df

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [64]:
# 모든 결측치를 0 대체
df.fillna(0)

,A,B,C,D
0,0.0,2,0.0,0
1,3.0,4,0.0,1
2,0.0,3,0.0,5
3,0.0,3,0.0,4


In [65]:
# 결측치를 이전행의 값으로 대체
df.fillna(method='ffill')

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [68]:
# 결측치를 다음행의 값으로 대체
df.fillna(method='bfill')

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [70]:
# 결측치를 열마다 특정값으로 대체
values = {'A':85, 'B':80, 'C':75, 'D':90}
df.fillna(value=values)

A    1
B    4
C    0
D    4
dtype: int64

In [60]:
# 결측치를 열별 평균값으로 대체
df.fillna(value=df.mean(axis=0))

NameError: name 'df' is not defined

In [185]:
# 결측치를 2번째 결측치 까지만 대체
df.fillna(value=values, limit=2)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,NaN,5
3,NaN,3,NaN,4


In [ ]:
# (1단계) iris 각 열마다 20개씩 인위적으로 결측치를 할당한다
# (2-1단계) 결측치는 각 열의 중위수로 대체한다 : fillna 사용
# (2-2단계) 결측치는 종별, 열별, 중위수(groupby, pivot_table)로 대체한다 : fillna 사용X

In [155]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.count()

Sepal.Length    150
Sepal.Width     150
Petal.Length    150
Petal.Width     150
Species         150
dtype: int64

In [156]:
# (1단계) iris 각 열마다 20개씩 인위적으로 결측치를 할당 후 확인
import random
import numpy as np
for col in range(iris.shape[1]-1) :
    iris.iloc[random.sample(range(150), 20), col] = np.nan

In [157]:
# 결측치 확인 - iris.isna() : 결측치 인지 여부
# iris.isna().sum()
iris.isnull().sum()

Sepal.Length    20
Sepal.Width     20
Petal.Length    20
Petal.Width     20
Species          0
dtype: int64

In [158]:
# 결측치가 하나라도 있는 행 출력
iris[iris['Sepal.Length'].isna() | iris['Sepal.Width'].isna() |
     iris['Petal.Length'].isna() | iris['Petal.Width'].isna()]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
2,4.7,NaN,1.3,NaN,setosa
5,NaN,3.9,1.7,0.4,setosa
6,NaN,NaN,NaN,NaN,setosa
7,5.0,NaN,1.5,0.2,setosa
11,4.8,3.4,NaN,0.2,setosa
...,...,...,...,...,...
136,NaN,3.4,5.6,NaN,virginica
138,6.0,NaN,4.8,1.8,virginica
141,6.9,NaN,NaN,2.3,virginica
143,6.8,NaN,5.9,2.3,virginica


In [159]:
# (2-1단계) 결측치는 각 열의 중위수로 대체한다 : fillna 사용
ex2 = iris.fillna(value=iris.mean(axis=0, numeric_only=True))
ex2.isna().sum()

Sepal.Length    0
Sepal.Width     0
Petal.Length    0
Petal.Width     0
Species         0
dtype: int64

In [160]:
# (2-2단계) 결측치는 종별, 열별, 중위수(groupby, pivot_table)로 대체한다 : fillna 사용X

In [161]:
iris_mean = iris.groupby(by="Species").mean()
iris_mean #, iris_mean.loc['setosa', 'Sepal.Length']

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,4.995349,3.375610,1.456818,0.240476
versicolor,5.947826,2.740909,4.262222,1.339130
virginica,6.560976,2.968889,5.570732,2.042857


In [197]:
# iris 한행을 매개변수로 받아 결측치가 있는 경우 종별 열별 중위수로 대체하는 함수
def iris_fillna(iris_row):
    iris_row = iris_row.copy()
    for col, value in iris_row.items():
        if isinstance(value, (int, float, np.number)) and np.isnan(value):
            iris_row[col] = iris_mean.loc[iris_row.Species, col]
    return iris_row

In [198]:
iris_fillna(iris.iloc[2])

Sepal.Length         4.7
Sepal.Width      3.37561
Petal.Length         1.3
Petal.Width     0.240476
Species           setosa
Name: 2, dtype: object

In [201]:
# apply 적용
iris = iris.apply(iris_fillna, axis=1)
iris

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.50000,1.4,0.200000,setosa
1,4.9,3.00000,1.4,0.200000,setosa
2,4.7,3.37561,1.3,0.240476,setosa
3,4.6,3.10000,1.5,0.200000,setosa
4,5.0,3.60000,1.4,0.200000,setosa
...,...,...,...,...,...
145,6.7,3.00000,5.2,2.300000,virginica
146,6.3,2.50000,5.0,1.900000,virginica
147,6.5,3.00000,5.2,2.000000,virginica
148,6.2,3.40000,5.4,2.300000,virginica


## 12.2 replace(to_value, new_value, inplace=T/F)
- to_value를 new_value
- to_value에 정규표현식 이용
- 데이터 프레임, 시리즈 적용

In [205]:
s = pd.Series([0,1,2,3,np.nan])
s.replace(np.nan, 0)
s.replace([0,1], 99) # 0 이나 1을 99로 변경
s.replace([0,1], [100, 101]) # 0은 100으로, 1은 101로 변경

0    100.0
1    101.0
2      2.0
3      3.0
4      NaN
dtype: float64

In [206]:
df = pd.DataFrame({'A': ['bat', 'foo', 'bait'],
                  'B': ['abc', 'bar', 'xyz']})
df.replace(r'ba.+', 'XX', regex=True)

,A,B
0,XX,abc
1,foo,XX
2,XX,xyz


## 12.3 mask(조건), where(조건)
- where(조건) : 조건이 True인 요소만 출력하고, 그 외 요소는 결측치
- mask(조건) : 조건이 False인 요소만 출력하고, 그 외 요소는 결측치

In [207]:
s = pd.Series(range(10, 20))
s

0    10
1    11
2    12
3    13
4    14
5    15
6    16
7    17
8    18
9    19
dtype: int64

In [208]:
s.where(s>15)

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6    16.0
7    17.0
8    18.0
9    19.0
dtype: float64

In [210]:
s.mask(s>15)

0    10.0
1    11.0
2    12.0
3    13.0
4    14.0
5    15.0
6     NaN
7     NaN
8     NaN
9     NaN
dtype: float64

## 12.4 dropna
- 결측치가 있는 데이터 누락(행, 열)

In [216]:
df = pd.DataFrame({'name':['홍','김','이', np.nan],
                   'age':[np.nan, 20, 30, np.nan],
                   'born':['서울','부산','인천',np.nan]})
df

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,인천
3,NaN,NaN,NaN


In [219]:
df.dropna(how='all', axis=0) # 모두 결측치인 행만 삭제

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,인천


In [220]:
df.dropna() # 결측치가 하나라도 있는 행 삭제

,name,age,born
1,김,20.0,부산
2,이,30.0,인천


In [221]:
df.dropna(axis=1) # 결측치가 하나라도 있는 열 삭제

""
0
1
2
3


In [224]:
df.dropna(subset=['name', 'born']) # 해당 열에 값이 있는 행들만 출력

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,인천


# 13절. 시리즈
- 1차원 데이터

In [228]:
s = pd.Series([1000, 7000, 5000],
             index=['mango', 'apple', 'banana'])
s

mango     1000
apple     7000
banana    5000
dtype: int64

In [229]:
s['mango'], s.mango

(1000, 1000)

In [232]:
s['mango'] = 2800 # 수정
s['orange'] = 4000 # 추가
s

mango     2800
apple     7000
banana    5000
orange    4000
dtype: int64

In [233]:
s['mango':'banana'] # 슬라이싱 [from:to] = from부터 to 포함

mango     2800
apple     7000
banana    5000
dtype: int64

In [235]:
s[ ['mango', 'banana']]

mango     2800
banana    5000
dtype: int64

In [237]:
s.drop('banana')

mango     2800
apple     7000
orange    4000
dtype: int64

In [240]:
s.sort_index()

mango     2800
orange    4000
banana    5000
apple     7000
dtype: int64

In [242]:
s.sort_values() # 오름차순 정렬

mango     2800
orange    4000
banana    5000
apple     7000
dtype: int64

In [246]:
s.sort_values(ascending=False) # 내림차순

apple     7000
banana    5000
orange    4000
mango     2800
dtype: int64

In [248]:
s.to_frame()
s.to_frame().T

,mango,apple,banana,orange
0,2800,7000,5000,4000


# 14절. 연습문제

In [244]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [245]:
# 1. iris 데이터에서 처음 다섯개 행만 출력하세요
iris.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [275]:
# 2. iris 데이터를 데이터프레임 변수인 독립변수 X와 종속변수 y로 나누세요. 종속변수는 species 열입니다.
X = iris[iris.columns[:-1]]
y = iris.species.to_frame()

,species
0,setosa
1,setosa
2,setosa
3,setosa
4,setosa
...,...
145,virginica
146,virginica
147,virginica
148,virginica


In [261]:
# 3. iris 데이터에서 처음 50개행을 빼내서 temp변수에 저장하세요
temp = iris.head(50)
temp

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


In [262]:
# 4. 3번에서 선택한 데이터프레임의 요약정보를 출력하세요. 모든 열에 대해 요약정보가 출력되어야 합니다
temp.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,50.00000,50.000000,50.000000,50.000000
mean,5.00600,3.428000,1.462000,0.246000
std,0.35249,0.379064,0.173664,0.105386
min,4.30000,2.300000,1.000000,0.100000
25%,4.80000,3.200000,1.400000,0.200000
50%,5.00000,3.400000,1.500000,0.200000
75%,5.20000,3.675000,1.575000,0.300000
max,5.80000,4.400000,1.900000,0.600000


In [263]:
# 5. versicolor종의 데이터만 iris_versicolor변수에 저장하세요
iris_versicolor = iris[iris.species == 'versicolor']
iris_versicolor

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
53,5.5,2.3,4.0,1.3,versicolor
54,6.5,2.8,4.6,1.5,versicolor
55,5.7,2.8,4.5,1.3,versicolor
56,6.3,3.3,4.7,1.6,versicolor
57,4.9,2.4,3.3,1.0,versicolor
58,6.6,2.9,4.6,1.3,versicolor
59,5.2,2.7,3.9,1.4,versicolor


In [282]:
# 6. 2번의 X와 y변수를 합해서 iris_df데이터 프레임으로 만드세요
import pandas as pd
pd.concat([X,y], axis=1)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [284]:
# 7. iris 데이터의 각 열 평균값을 출력하세요
iris.mean(numeric_only=True)

sepal_length    5.843333
sepal_width     3.057333
petal_length    3.758000
petal_width     1.199333
dtype: float64

In [288]:
# 8. iris 데이터의 각 열들 사이의 상관계수를 출력하세요.
iris.corr(numeric_only=True)

,sepal_length,sepal_width,petal_length,petal_width
sepal_length,1.000000,-0.117570,0.871754,0.817941
sepal_width,-0.117570,1.000000,-0.428440,-0.366126
petal_length,0.871754,-0.428440,1.000000,0.962865
petal_width,0.817941,-0.366126,0.962865,1.000000


In [402]:
# 9. iris 데이터에서 각 요소들과 변수별 평균과의 차이를 출력하세요.
# iris.apply(lambda x : x[:-1]-iris.mean(numeric_only=True), axis=1)
iris_mean = iris.iloc[:,:-1].mean()
iris_mean

sepal_length    5.843333
sepal_width     3.057333
petal_length    3.758000
petal_width     1.199333
dtype: float64

In [306]:
# 10. iris 데이터의 종별 평균을 출력하세요
iris.groupby(iris.species).mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [356]:
# 11.iris 데이터에서 각 요소들과 종별 변수의 평균과의 차이를 출력하세요. 종별로 3개씩 출력.
species_mean = iris.groupby(iris.species).mean()
def iris_avg(iris_row) :
    iris_row = iris_row.copy()
    for col, value in iris_row.items() :
        if isinstance(value, (int, float, np.number)):
            avg = species_mean.loc[iris_row.species, col]
            iris_row[col] = value - avg
    return iris_row

In [399]:
iris.apply(iris_avg, axis=1).groupby(iris.species).take(random.sample(range(50), 3)).sort_index()

sepal_length  sepal_width  petal_length  petal_width     species
species                                                                         
setosa     10          0.394        0.272         0.038       -0.046      setosa
           15          0.694        0.972         0.038        0.154      setosa
           21          0.094        0.272         0.038        0.154      setosa
versicolor 60         -0.936       -0.770        -0.760       -0.326  versicolor
           65          0.764        0.330         0.140        0.074  versicolor
           71          0.164        0.030        -0.260       -0.026  versicolor
virginica  110        -0.088        0.226        -0.452       -0.026   virginica
           115        -0.188        0.226        -0.252        0.274   virginica
           121        -0.988       -0.174        -0.652       -0.026   virginica

In [403]:
# 방법1
iris_mean_by_species = iris.groupby(iris.species).mean()
iris_mean_by_species

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [404]:
iris.loc[0] - iris_mean_by_species.loc[iris.loc[0].species]

petal_length   -0.062
petal_width    -0.046
sepal_length    0.094
sepal_width     0.072
species           NaN
dtype: object

In [409]:
result = iris.apply(lambda series : series-iris_mean_by_species.loc[series.species], axis=1)
result.groupby(iris.species).sample(3).sort_index()

,petal_length,petal_width,sepal_length,sepal_width,species
14,-0.262,-0.046,0.794,0.572,NaN
33,-0.062,-0.046,0.494,0.772,NaN
41,-0.162,0.054,-0.506,-1.128,NaN
68,0.240,0.174,0.264,-0.570,NaN
92,-0.260,-0.126,-0.136,-0.170,NaN
97,0.040,-0.026,0.264,0.130,NaN
121,-0.652,-0.026,-0.988,-0.174,NaN
136,0.048,0.374,-0.288,0.426,NaN
148,-0.152,0.274,-0.388,0.426,NaN


In [418]:
# 방법2
iris_grouped_sample = iris.groupby('species').head(3)
iris_grouped_sample.apply(lambda temp:temp - iris_mean_by_species.loc[temp.species], 
                         axis=1)

,petal_length,petal_width,sepal_length,sepal_width,species
0,-0.062,-0.046,0.094,0.072,NaN
1,-0.062,-0.046,-0.106,-0.428,NaN
2,-0.162,-0.046,-0.306,-0.228,NaN
50,0.440,0.074,1.064,0.430,NaN
51,0.240,0.174,0.464,0.430,NaN
52,0.640,0.174,0.964,0.330,NaN
100,0.448,0.474,-0.288,0.326,NaN
101,-0.452,-0.126,-0.788,-0.274,NaN
102,0.348,0.074,0.512,0.026,NaN
